In [1]:
import pandas as pd
pd.options.display.max_colwidth = 10
traindf=[]
traindf = pd.read_csv('labeledTrainData.tsv', delimiter="\t")
traindf = traindf.drop(['id'], axis=1)
traindf['type'] = 'train'
addtldf=[]
addtldf = pd.read_csv('unlabeledTrainData.tsv',error_bad_lines=False,delimiter="\t")
addtldf = addtldf.drop(['id'], axis=1)
addtldf['type'] = 'addtl'
revcorpus = pd.concat([traindf,addtldf],ignore_index=True)
print("Number of reviews: ",len(revcorpus))

b'Skipping line 43043: expected 2 fields, saw 3\n'


Number of reviews:  74998


C:\Users\Ellysah\Anaconda3\envs\keras-gpu\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # This is added back by InteractiveShellApp.init_path()


In [2]:
import re
import string
# import the inflect library 
import inflect 
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.stem.porter import PorterStemmer 
from sklearn.feature_extraction.text import CountVectorizer
from nltk.util import ngrams,bigrams
from nltk.tokenize import sent_tokenize

stemmer = PorterStemmer() 
p = inflect.engine() 
stop_words = set(stopwords.words("english")) 
# extract pure text from html
def remove_tags(text):
    text = re.sub(r'<[^<>]+>', " ", text)
    return text
# remove stopwords function 
def remove_stopwords(text): 
    word_tokens = word_tokenize(text) 
    filtered_text = [word for word in word_tokens if word not in stop_words] 
    filtered_text = " ".join(filtered_text)
    return filtered_text 

def text_lowercase(text): 
    return text.lower() 
  
# convert number into words 
def convert_number(text): 
    # split string into list of words 
    temp_str = text.split() 
    # initialise empty list 
    new_string = [] 
  
    for word in temp_str: 
        # if word is a digit, convert the digit 
        # to numbers and append into the new_string list 
        if word.isdigit(): 
            temp = p.number_to_words(word) 
            new_string.append(temp) 
  
        # append the word as it is 
        else: 
            new_string.append(word) 
  
    # join the words of new_string to form a string 
    temp_str = ' '.join(new_string) 
    return temp_str

# remove punctuation 
def remove_punctuation(text): 
    translator = str.maketrans('', '', string.punctuation) 
    return text.translate(translator)

    
# remove whitespace from text 
def remove_whitespace(text): 
    return  " ".join(text.split()) 

lemmatizer = WordNetLemmatizer() 
# lemmatize string 
def lemmatize_word(text): 
    word_tokens = word_tokenize(text) 
    # provide context i.e. part-of-speech 
    lemmas = [lemmatizer.lemmatize(word, pos ='v') for word in word_tokens] 
    lemmas = " ".join(lemmas)
    return lemmas 
# stem words in the list of tokenised words 
def stem_words(text): 
    word_tokens = word_tokenize(text) 
    stems = [stemmer.stem(word) for word in word_tokens] 
    stems = " ".join(stems)
    return stems 

def generate_ngram(text,n):
    text = re.sub(r'[^a-zA-Z0-9\s]', ' ', text)
    tokens = word_tokenize(text) 
    output = list(ngrams(tokens, n))
    #output = bigrams(tokens)
    return output

def preprocess_text(text):
    text = remove_tags(text)
    text = convert_number(text)
    text = text_lowercase(text)
    text = remove_punctuation(text)
    text = remove_stopwords(text)
    text = remove_whitespace(text)
    #text = stem_words(text)
    #text = ngram_vector(text)
    #print(text)
    text = lemmatize_word(text)
    return text
#revcorpus = revcorpus[1:5]
revcorpus['processed_review'] = revcorpus.review.apply(lambda x: preprocess_text(x))  
#revcorpus['bigrams'] = revcorpus.processed_review.apply(lambda x:generate_ngram(x,2))
print("Number of reviews: ",len(revcorpus))
print(revcorpus.head(3))
#traindf['processed_review'] = traindf.review.apply(lambda x: preprocess_text(x))  
#print("Number of reviews: ",len(traindf))
#traindf.head(3)

Number of reviews:  74998
      review  sentiment   type processed_review
0  With a...        1.0  train  stuff ...      
1  \The C...        1.0  train  classi...      
2  The fi...        0.0  train  film s...      


In [3]:
from gensim.models import Phrases
reviews = revcorpus.processed_review.apply(lambda x: word_tokenize(x))  
bigrams = Phrases(sentences=reviews)
print(bigrams[reviews][2])

['film', 'start', 'manager', 'nicholas', 'bell', 'give', 'welcome', 'investors', 'robert_carradine', 'primal', 'park', 'secret', 'project', 'mutate', 'primal', 'animal', 'use', 'fossilize', 'dna', 'like', '¨jurassik', 'park¨', 'scientists', 'resurrect', 'one', 'natures', 'fearsome', 'predators', 'sabretooth', 'tiger', 'smilodon', 'scientific', 'ambition', 'turn', 'deadly', 'however', 'high_voltage', 'fence', 'open', 'creature', 'escape', 'begin', 'savagely', 'stalk_prey', 'human', 'visitors', 'tourists', 'scientificmeanwhile', 'youngsters', 'enter', 'restrict_area', 'security', 'center', 'attack', 'pack', 'large', 'prehistorical', 'animals', 'deadlier', 'bigger', 'addition', 'security', 'agent', 'stacy', 'haiduk', 'mate', 'brian', 'wimmer', 'fight', 'hardly', 'carnivorous', 'smilodons', 'sabretooths', 'course', 'real', 'star', 'star', 'astound', 'terrifyingly', 'though', 'convince', 'giant', 'animals', 'savagely', 'stalk_prey', 'group', 'run_afoul', 'fight', 'one', 'natures', 'fearsome

In [4]:
reviews = bigrams[reviews]
reviews = [" ".join(text) for text in [review for review in reviews]]

In [5]:
reviews[1]

'classic war_worlds timothy_hines entertain film obviously go great effort lengths faithfully recreate h_g well classic book mr_hines succeed watch film appreciate fact standard predictable hollywood_fare come every year eg spielberg version tom_cruise slightest resemblance book obviously everyone look different things movie envision amateur critics look criticize everything others rate movie important baseslike entertain people never agree critics enjoy effort mr_hines put faithful hg_well classic_novel find entertain make easy_overlook critics perceive shortcomings'

In [6]:
#BAG OF WORDS MODEL
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
max_features = 3000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(reviews)
list_tokenized_train = tokenizer.texts_to_sequences(reviews)
maxlen = 200
X_pad = pad_sequences(list_tokenized_train, maxlen=maxlen)

Using TensorFlow backend.


In [7]:
X_pad[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,  423,   11,  478,  121,   82,  972,
        145,   12,  931,  551,   12,   12,  193,   46,    7,  650, 2230,
         70,   15,   16,  508,  193,    4,  227,  662, 2134, 1168,   58,
         58,  271,    1,  245,   11,    6,  372, 1588,  276, 1174,  559,
         59,  775, 2026,   27,  483,  559,  715,   26, 1904, 1032,  187,
        456, 1734,  816, 2138,    5,  485,   11,  654,   38,  176,  119,
        169,    4,    2,  114,   14,   23,    4,  114,  206,   16,  258,
        727,  271,    1,  115,  347,   82,  652,  147, 1496,  317,  808,
        529,  801,  715, 1356,   46,  263,   26,  583,  517,  808,   10,
         46,   25,   24, 2671,  715,  452,  193,  743,  145,   64,  508,
         97,    5,   92,  437, 1990,  135, 1602, 2450,  317,   27,   91,
        134,   37,    1,   26,  317,  523,  598,  654,   44,    3,  141,
        163,  524,  135,  628,  929, 1112,  458,   

In [8]:
X_pad = X_pad[0:25000]
y = revcorpus[revcorpus['type']=="train"]['sentiment']

In [9]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten,CuDNNLSTM, CuDNNGRU
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
from keras import initializers, regularizers, constraints, optimizers, layers

embed_size = 200
model = Sequential()
#model.add(Embedding(input_dim=wordvec_model.wv.vectors.shape[0],output_dim=wordvec_model.wv.vectors.shape[1],weights=[wordvec_model.wv.vectors], input_length=embed_size))
model.add(Embedding(max_features,embed_size))
model.add(CuDNNLSTM(64,return_sequences = True))
model.add(GlobalMaxPool1D())
model.add(Dropout(0.1))
model.add(Dense(20,activation='relu'))
model.add(Dropout(0.05))
model.add(Dense(1, activation="sigmoid"))
model.summary()



W1005 11:41:00.244666  9960 deprecation_wrapper.py:119] From C:\Users\Ellysah\Anaconda3\envs\keras-gpu\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1005 11:41:00.262618  9960 deprecation_wrapper.py:119] From C:\Users\Ellysah\Anaconda3\envs\keras-gpu\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1005 11:41:00.264613  9960 deprecation_wrapper.py:119] From C:\Users\Ellysah\Anaconda3\envs\keras-gpu\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1005 11:41:01.101415  9960 deprecation_wrapper.py:119] From C:\Users\Ellysah\Anaconda3\envs\keras-gpu\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_w

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 200)         600000    
_________________________________________________________________
cu_dnnlstm_1 (CuDNNLSTM)     (None, None, 64)          68096     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 64)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 20)                1300      
_________________________________________________________________
dropout_2 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 21        
Total para

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_pad, y, test_size=0.2, shuffle=True)
X_test

array([[   0,    0,    0, ...,   11,   63,   76],
       [   0,    0,    0, ...,  549,  126,  203],
       [   0,    0,    0, ...,   19,  290, 2329],
       ...,
       [   0,    0,    0, ...,    2,  209,    8],
       [   0,    0,    0, ...,  905,  147,   63],
       [   0,    0,    0, ...,    1,  316,   84]])

In [13]:
batch_size = 32
epochs = 5
model.compile(loss='binary_crossentropy', optimizer='AdaGrad', metrics=['accuracy'])
model.fit(X_train,y_train, batch_size=batch_size, epochs=epochs)

Epoch 1/5
20000/20000 [==============================] - 14s 685us/step - loss: 0.2268 - acc: 0.9110
Epoch 2/5
20000/20000 [==============================] - 13s 661us/step - loss: 0.1370 - acc: 0.9519
Epoch 3/5
20000/20000 [==============================] - 13s 661us/step - loss: 0.1050 - acc: 0.9673
Epoch 4/5
20000/20000 [==============================] - 13s 660us/step - loss: 0.0836 - acc: 0.9750
Epoch 5/5
20000/20000 [==============================] - 13s 663us/step - loss: 0.0674 - acc: 0.9813


In [14]:
# validate the model on test dataset to determine generalization
loss, acc = model.evaluate(X_test, y_test, batch_size=batch_size)
print("\nTest accuracy: %.1f%%" % (100.0 * acc))

5000/5000 [==============================] - 2s 301us/step

Test accuracy: 85.9%
